In [19]:
import os
import time
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision

In [10]:
seed = 1234
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [11]:
# DataLoaderの作成
from utils.dataloader import make_datapath_list, DataTransform, COCOkeypointsDataset

train_img_list, train_mask_list, val_img_list, val_mask_list, train_meta_list, val_meta_list = make_datapath_list(rootpath='./data/')

train_dataset = COCOkeypointsDataset(val_img_list, val_mask_list, val_meta_list, phase='train',
                                    transform=DataTransform())

batch_size = 12

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataloaders_dict = {'train': train_dataloader, 'val': None}

In [12]:
from utils.openpose_net import OpenPoseNet
net = OpenPoseNet()

In [22]:
# 損失関数の定義
class OpenPoseLoss(nn.Module):
    def __init__(self):
        super(OpenPoseLoss, self).__init__()
        
    def forward(self, saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask):
        '''
        Parameters
        ----------
        saved_for_loss: OpenPoseNetの出力list
        heatmap_target: [num_batch, 19, 46, 46]
        paf_target: [num_batch, 38, 46, 46]
        paf_mask: [num_batch, 38, 46, 46]
        
        Returns
        -------
        loss: テンソル 損失の値
        '''
        total_loss = 0
        # stageごとに計算
        for j in range(6):
            #PAFs
            pred1 = saved_for_loss[2 * j] * paf_mask
            gt1 = paf_target.float() * paf_mask
            
            # heatmaps
            pred2 = saved_for_loss[2 * j + 1] * heat_mask
            gt2 = heatmap_target.float() * heat_mask
            
            total_loss += F.mse_loss(pred1, gt1, reduction='mean') + \
                F.mse_loss(pred2, gt2, reduction='mean')
            
        return total_loss

In [23]:
criterion = OpenPoseLoss()

In [24]:
optimizer = optim.SGD(net.parameters(), lr=1e-2, momentum=0.9, weight_decay=0.0001)

In [27]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    num_train_imgs = len(dataloaders_dict['train'].dataset)
    batch_size = dataloaders_dict['train'].batch_size
    
    iteration = 1
    
    for epoch in range(num_epochs):
        t_epoch_start = time.time()
        t_iter_start = time.time()
        epoch_train_loss = 0
        epoch_val_loss = 0
        
        print(f'Epoch {epoch+1}/{num_epochs}')
        
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
                optimizer.zero_grad()
                print(' (train) ')
            else:
                continue # 今回はvalはスキップ
            
            for imgs, heatmap_target, heat_mask, paf_target, paf_mask in dataloaders_dict[phase]:
                if imgs.size()[0] == 1:
                    continue
                
                imgs = imgs.to(device)
                heatmap_target = heatmap_target.to(device)
                heat_mask = heat_mask.to(device)
                paf_target = paf_target.to(device)
                paf_mask = paf_mask.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    # (out6_1, out6_2)は使わないので_ で代替
                    _, saved_for_loss = net(imgs)
                    
                    loss = criterion(saved_for_loss, heatmap_target, heat_mask, paf_target, paf_mask)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        if (iteration % 10 == 0):
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            print(f'iter {iteration} || Loss: {loss.item()/batch_size:.4f} || 10iter: {duration:.4f} sec.')
                            t_iter_start = time.time()
                            
                        epoch_train_loss += loss.item()
                        iteration += 1
                            
        t_epoch_finish = time.time()
        print(f'epoch {epoch+1} || epoch_train_loss: {epoch_train_loss/num_train_imgs:.4f} || epoch_val_loss: {0}')
        print(f'timer: {t_epoch_finish - t_epoch_start:.4f} sec.')
        t_epoch_start = time.time()
        
    torch.save(net.state_dict(), 'weights/openpose_net_' + str(epoch+1) + '.pth')

In [28]:
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/2
 (train) 
iter 10 || Loss: 0.0035 || 10iter: 11.1171 sec.
iter 20 || Loss: 0.0035 || 10iter: 11.1181 sec.
iter 30 || Loss: 0.0043 || 10iter: 11.0731 sec.
iter 40 || Loss: 0.0034 || 10iter: 11.2403 sec.
iter 50 || Loss: 0.0047 || 10iter: 10.9861 sec.
iter 60 || Loss: 0.0050 || 10iter: 11.6048 sec.
iter 70 || Loss: 0.0036 || 10iter: 10.7852 sec.
iter 80 || Loss: 0.0038 || 10iter: 11.0327 sec.
iter 90 || Loss: 0.0050 || 10iter: 10.2230 sec.
iter 100 || Loss: 0.0038 || 10iter: 11.4267 sec.
iter 110 || Loss: 0.0046 || 10iter: 11.6475 sec.
iter 120 || Loss: 0.0031 || 10iter: 11.1521 sec.
iter 130 || Loss: 0.0046 || 10iter: 10.5557 sec.
iter 140 || Loss: 0.0048 || 10iter: 11.4457 sec.
iter 150 || Loss: 0.0038 || 10iter: 11.1559 sec.
iter 160 || Loss: 0.0049 || 10iter: 10.5505 sec.
iter 170 || Loss: 0.0029 || 10iter: 11.1427 sec.
iter 180 || Loss: 0.0033 || 10iter: 12.5570 sec.
iter 190 || Loss: 0.0032 || 10iter: 10.4932 sec.
iter 200 || Loss: 0.0034 || 10iter: 11.0702 sec.
iter 210 